In [36]:
'''
获取鲁菜、苏菜的菜谱信息 包括菜品名称 图片 主辅调料（用量）以及制作步骤
'''
import requests
import lxml
from bs4 import BeautifulSoup
import time
import random
import re
import csv
import os

user_agents = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]

# 随机选取请求头中的 user-agent
request_headers={
    'user-agent':random.choice(user_agents),
    'Referer': "https://www.douban.com"
}


In [37]:
def get_per_page(url,dishes,m_ingredients,s_ingredients,c_ingredients,ingre_amount,dish_type):

    web_data = requests.get(url=url,headers=request_headers)
    status_code = web_data.status_code
    if status_code == 200:
        html_text =web_data.text
        # 解析为soup文档
        soup_document = BeautifulSoup(html_text,'html.parser')
        # 定位到所有菜品超链的 a标签
        dish_homepage_a_elements = soup_document.select('div.detail > h2 > a')
        # 定位到所有的菜品图片
        image_tags = soup_document.select('#J_list > ul > li > div.pic > a > img')

        # 遍历每一道菜品s
        for (dish_homepage_a_element,image_tag) in zip(dish_homepage_a_elements,image_tags):
            #time.sleep(random.uniform(0.2,0.4))
            # 提取 href 属性值
            dish_homepage_href = dish_homepage_a_element.get('href').strip()
            #print(dish_homepage_href)
            # 获取每一道菜品详情页中的信息
            dish,m_ingredient,s_ingredient,c_ingredient,ingredient_amount = get_dish_info(dish_homepage_href)
            
            # 添加菜系的属性
            dish.append(dish_type)
            dishes.append(dish)
            m_ingredients.append(m_ingredient)
            
            for ele in ingredient_amount:
                ingre_amount.append(ele)
            
            
            a = 0
            # 将用量、原料和菜品对应起来
            for ele in s_ingredient:
                s_ingredients.append(ele)
                a +=1
         
            if c_ingredients != '':
                b = 0
                for ele in c_ingredient:
                    c_ingredients.append(ele)
                    b +=1
                ingredients = [m_ingredients[-1]]+s_ingredients[-a:-1]+c_ingredients[-b:-1]
            else:
                ingredients = [m_ingredients[-1]]+s_ingredients[-a:-1]        

            for i in range(0,len(ingredients)):
                ingre_amount_l.append([ingredients[i],ingre_amount[i],dish[0]])
            

            '''
            获取 每道菜的图片 并以菜名来命名保存至本地
            '''
            '''
            image_url = image_tag.get('data-src')  # 获取图片链接
            print(image_url)

            # 发起请求下载图片
            time.sleep(random.uniform(0,0.4))
            image_response = requests.get(image_url,headers=request_headers)
            status_code = image_response.status_code
            if image_response.status_code == 200:
                # 获取图片的文件名，可以根据需要自行修改保存路径和文件名
                file_name = dish[0]
                file_name = '%s.jpg'%file_name

                file_path = './picture' 
                if not os.path.exists(file_path):
                    os.mkdir(file_path)

                # 保存图片到本地
                with open(file_path + os.sep + file_name, 'wb') as f:
                    f.write(image_response.content)
                    print(f"图片 {file_name} 保存成功！")
            else:
                print(status_code)
            '''

def get_dish_info(dish_homepage_href):

    web_data = requests.get(url=dish_homepage_href,headers=request_headers)
    status_code = web_data.status_code
    if status_code == 200:
        web_data.encoding = 'utf-8'
        html_text = web_data.text
        soup = BeautifulSoup(html_text,'lxml') 
        selector = lxml.etree.HTML(html_text)
        
        '''
        获取 dish 表中的内容
        '''
        dish_name_elements = soup.select('div.userTop.clear')

        dish_feature_elements = soup.select('span.category_s1')
        dish_taste_elements = [dish_feature_elements[-4]]
        dish_method_elements = [dish_feature_elements[-3]]
        dish_time_cousuming_elements = [dish_feature_elements[-2]]
        dish_difficulty_elements = [dish_feature_elements[-1]]

        dish_steps_elements = soup.select('div.recipeStep_word')

        for dish_name_element,dish_taste_element,dish_method_element,dish_time_cousuming_element,dish_difficulty_element,dish_step_element in zip(
            dish_name_elements,dish_taste_elements,dish_method_elements,dish_time_cousuming_elements,dish_difficulty_elements,dish_steps_elements):

            dish_name = dish_name_element.find('a').get('title').strip()             
            dish_taste = dish_taste_element.find('a').get_text().strip()
            dish_method = dish_method_element.find('a').get_text().strip()
            dish_time_consuming = dish_time_cousuming_element.find('a').get_text().strip()
            dish_difficulty = dish_difficulty_element.find('a').get_text().strip()
            
            dish_steps = ''
            for i in range(len(dish_steps_elements)):
                dish_steps += dish_steps_elements[i].get_text().strip()

            #print(dish_name,dish_taste,dish_method,dish_time_consuming,dish_difficulty,dish_steps)
            
            dish = [dish_name,dish_taste,dish_method,dish_time_consuming,dish_difficulty,dish_steps]
        

        '''
         获取 main_ingredient secondary_ingredient condiment 表中的内容
        '''
        main_ingre_elements = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li/span[1]/b')
        if not main_ingre_elements:
            main_ingre_elements = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li/span[1]/a/b')
        for i in range(0,len(main_ingre_elements)):
            time.sleep(random.uniform(0,0.4))
            main_ingredient_name = main_ingre_elements[i].text
            
        
        secondary_ingre_elements = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[2]/div/ul/li/span[1]/a/b')
        secondary_ingredient_name = []
        for i in range(0,len(secondary_ingre_elements)):
            time.sleep(random.uniform(0,0.3))
            secondary_ingredient_name.append(secondary_ingre_elements[i].text)

        condiment_elements = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[3]/div/ul/li/span[1]/a/b')
        if not condiment_elements:
            condiment_name = '' 
        else:
            condiment_name = []    
            for i in range(0,len(condiment_elements)):
                time.sleep(random.uniform(0,0.2))
                condiment_name.append(condiment_elements[i].text)

        ingre_amount_elements = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset/div/ul/li/span[2]')
        ingredient_amount = []
        for i in range(0,len(ingre_amount_elements)):
            time.sleep(random.uniform(0,0.2))
            ingredient_amount.append(ingre_amount_elements[i].text)
        #print(ingredient_amount)
        
        return dish,main_ingredient_name,secondary_ingredient_name,condiment_name,ingredient_amount

In [38]:
def save_to_csv(dishes,m_ingredients,s_ingredients,c_ingredients,ingre_amount_l):

    '''
    with open('dish.csv', 'w', encoding='utf-8') as f:
        f.write('菜品名,口味,工艺,耗时,难度,步骤\n')
        for dish in dishes:
            #print(dish)
            row = '{},{},{},{},{},{}'.format(dish[0], dish[1],dish[2],dish[3],dish[4],dish[5])
            f.write(row)
            f.write('\n')

    with open('main_ingredients.csv', 'w', encoding='utf-8') as f:
        f.write('主料名\n')
        for m_ingredient in m_ingredients:
            row = m_ingredient
            f.write(row)
            f.write('\n')
    
    with open('secondary_ingredients.csv', 'w', encoding='utf-8') as f:
        f.write('辅料名\n')
        for s_ingredient in s_ingredients:
            row = s_ingredient
            f.write(row)
            f.write('\n')
    
    with open('condiment.csv', 'w', encoding='utf-8') as f:
        f.write('调料名\n')
        for c_ingredient in c_ingredients:
            row = c_ingredient
            f.write(row)
            f.write('\n')
    
    with open('cuisine_dish.csv', 'w', encoding='utf-8') as f:
        f.write('菜品名,关系,菜系\n')
        for dish in dishes:
            row = '{},属于,{}'.format(dish[0],dish[6])
            f.write(row)
            f.write('\n')
    '''
    with open('ingre_amount_l.csv', 'w', encoding='utf-8') as f:
        f.write('原料名,用量,菜品名\n')
        for ingre_amount in ingre_amount_l:
            row = '{},{},{}'.format(ingre_amount[0],ingre_amount[1],ingre_amount[2])
            f.write(row)
            f.write('\n')
    

In [39]:

urls_sucai = ['https://home.meishichina.com/recipe/sucai/page/{}/'.format(str(no)) for no in range(1,4)]
urls_lucai = ['https://home.meishichina.com/recipe/lucai/page/{}/'.format(str(no)) for no in range(1,4)]

dishes = []
m_ingredients = []
s_ingredients = []
c_ingredients = []
ingre_amount_l = []
ingre_amount = []

for url in urls_sucai:
    get_per_page(url,dishes,m_ingredients,s_ingredients,c_ingredients,ingre_amount,'苏菜')
    print(ingre_amount)
    time.sleep(random.uniform(0,0.6))

for url in urls_lucai:
    get_per_page(url,dishes,m_ingredients,s_ingredients,c_ingredients,ingre_amount,'鲁菜')
    print(ingre_amount)
    time.sleep(random.uniform(0,0.6))



# 经典的python去重 且利用列表递推式去除'' 即没有调料的特殊处理
#c_ingredients = list(set([item for item in c_ingredients if item != '']))
#s_ingredients = list(set([item for item in s_ingredients if item != '']))
#print(len(c_ingredients))
#print(len(s_ingredients))

save_to_csv(dishes,m_ingredients,s_ingredients,c_ingredients,ingre_amount_l)

print('End--------')

['500克', '4小块', '1块', '25克', '少许', '少许', '7个', '适量', '1碗', '2勺', '2勺', '1勺', '1块', '1个', '3片', '少许', '1棵', '1000g', '250ml', '250g', '适量', '适量', '适量', '适量', '适量', '适量', '适量', '适量', '适量', '适量', '150克', '2根', '150克', '2克', '适量', '300g', '8瓣', '一匙', '少许', '少许', '少许', '适量', '适量', '适量', '适量', '少许', '1000克', '1粒', '半勺', '半勺', '2茶匙', '半勺', '15克', '2颗', '1块', '1片', '1小块', '1个', '250g', '140g', '7g', '2g', '3g', '1.5g', '180g', '90g', '1把', '1大勺', '1小勺', '适量', '适量', '适量', '100g', '25g', '30g', '20g', '150g', '40g', '245g', '数张', '2个', '半个', '半个', '1个', '数个', '适量', '适量', '适量', '适量', '适量', '适量', '适量', '2颗', '2根', '3-4瓣', '适量', '400克', '两茶匙', '三茶匙', '两茶匙', '三分之一茶匙', '适量', '适量', '三分之一茶匙', '适量']
['500克', '4小块', '1块', '25克', '少许', '少许', '7个', '适量', '1碗', '2勺', '2勺', '1勺', '1块', '1个', '3片', '少许', '1棵', '1000g', '250ml', '250g', '适量', '适量', '适量', '适量', '适量', '适量', '适量', '适量', '适量', '适量', '150克', '2根', '150克', '2克', '适量', '300g', '8瓣', '一匙', '少许', '少许', '少许', '适量', '适量', '适量', '适量', '少许', '1000克', '1粒',